## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-08-48-20 +0000,bbc,Reeves disappointed after economy unexpectedly...,https://www.bbc.com/news/articles/cq6mvem8neno
1,2025-07-11-08-36-26 +0000,nypost,Legendary California seafood restaurant clashe...,https://nypost.com/2025/07/11/us-news/californ...
2,2025-07-11-08-32-32 +0000,bbc,MP Rupert Lowe investigated by standards watchdog,https://www.bbc.com/news/articles/ceq7z421pwzo
3,2025-07-11-08-31-00 +0000,wsj,"ECB’s Bar for Another Rate Cut Is High, Schnab...",https://www.wsj.com/articles/ecbs-bar-for-anot...
4,2025-07-11-08-28-20 +0000,bbc,Plans for cash Isa changes on hold after backlash,https://www.bbc.com/news/articles/cqjq9yxkkrvo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,14
164,tariffs,6
60,uk,6
214,where,4
65,against,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
52,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,40
66,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,38
28,2025-07-11-00-52-06 +0000,nypost,Hochul blames Trump’s ‘Big Beautiful’ bill for...,https://nypost.com/2025/07/10/us-news/hochul-b...,30
24,2025-07-11-02-31-43 +0000,nypost,Trump admin still deciding where it would depo...,https://nypost.com/2025/07/10/us-news/trump-ad...,29
10,2025-07-11-06-22-59 +0000,nypost,Woman with ‘urgent message’ for President Trum...,https://nypost.com/2025/07/11/us-news/florida-...,29


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
52,40,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
6,25,2025-07-11-08-22-06 +0000,nypost,Georgia teen Sarah Grace Patrick accused of ki...,https://nypost.com/2025/07/11/us-news/georgia-...
7,21,2025-07-11-08-06-08 +0000,bbc,UK-France migrant deal 'robust' against legal ...,https://www.bbc.com/news/articles/cx24d70gw41o
21,20,2025-07-11-03-35-21 +0000,nypost,Grieving mom of dead Camp Mystic camper desper...,https://nypost.com/2025/07/10/us-news/grieving...
30,16,2025-07-11-00-45-44 +0000,nypost,Search-and-rescue volunteers had to rely on se...,https://nypost.com/2025/07/10/us-news/search-a...
28,16,2025-07-11-00-52-06 +0000,nypost,Hochul blames Trump’s ‘Big Beautiful’ bill for...,https://nypost.com/2025/07/10/us-news/hochul-b...
26,16,2025-07-11-01-57-20 +0000,nypost,Florida scammer posing as doctor gave Botox in...,https://nypost.com/2025/07/10/us-news/florida-...
20,14,2025-07-11-04-01-24 +0000,nypost,Protester fires gun at federal agents in chaot...,https://nypost.com/2025/07/11/us-news/californ...
27,14,2025-07-11-01-41-11 +0000,nypost,"NYPD, other city agencies crack down on trash,...",https://nypost.com/2025/07/10/us-news/nyc-crac...
29,14,2025-07-11-00-46-14 +0000,nypost,Largest US teachers union mocked for misspelli...,https://nypost.com/2025/07/10/us-news/largest-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
